In [8]:
import numpy as np
from numpy.linalg import norm
import pandas as pd
import itertools
import random
from matplotlib import pyplot as plt
from pair_trading_foundations.data_generation import ExecutePairTrading, generate_training_data
random.seed(23)
import cProfile
import pstats
import pickle
import plotly.express as px
from time import time

def chunker(seq, size):
    # split a list into chunks
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [15]:
# data = pd.read_csv('Data/sp500_full_20181231_to_20231229.csv')
data = pd.read_csv('Data/sp500_full_20150101_to_20191231.csv')

In [16]:
value_count_tb = data[['Ticker']].groupby('Ticker').size().reset_index()
value_count_tb.columns = ['Ticker', 'Count']
stock_to_keep = value_count_tb['Ticker'][value_count_tb.Count==value_count_tb.Count.max()]
data = data[data.Ticker.isin(stock_to_keep)]
sampled_tickers = random.sample(list(stock_to_keep.values), 10)
# data_tech = data[data['GICS Sector'].isin(['Information Technology'])]
data_sampled = data[data['Ticker'].isin(sampled_tickers)]

In [17]:
tickers = list(set(data.Ticker.values))
combinations = list(itertools.combinations(tickers, 2))

In [18]:
len(combinations)

113050

In [19]:
batches = list(chunker(combinations, 1000))

# Generate for all pairs

In [20]:
i = 0
for batch in batches:
    start_ts=time()
    print(f'Getting {i+1}th out of {len(batches)} batches')
    features_tb, labels_tb, pnl_metadata_tb = generate_training_data(
        data=data,
        training_len=300,
        test_len=120,
        calculate_label=True,
        verbose=False,
        combinations=batch
    )
    combined = pd.merge(features_tb, labels_tb, how='left', on=['Date', 'Ticker_P1','Ticker_P2']).reset_index(drop=True)
    # combined = pd.merge(combined, pnl_metadata_tb[['Date', 'Ticker_P1','Ticker_P2', 'trade_executions']], how='left', on=['Date', 'Ticker_P1','Ticker_P2'])
    # combined = combined[combined.pnls.notnull()].reset_index(drop=True)
    combined.to_csv(f'Data/Training/pair_features{i+1}_300_120.csv', index=False)
    end_ts = time()
    print(f"Took {end_ts - start_ts} seconds")
    i+=1
    break

Getting 1th out of 114 batches
1000 stock pairs detected
Took 0.15260100364685059 to initilize. Entering ticker pair loop


/Users/xiaoma/Desktop/current_desktop/mids/pair_trading/pair-trading-foundations/notebooks/pair_trading_foundations/data_generation.py:227: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  features_tb = pd.concat(
/Users/xiaoma/Desktop/current_desktop/mids/pair_trading/pair-trading-foundations/notebooks/pair_trading_foundations/data_generation.py:273: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  labels_tb = pd.concat(
/Users/xiaoma/Desktop/current_desktop/mids/pair_trading/pair-trading-foundations

Getting the 100th pair
Used 312.9467430114746 for the 100 pairs


KeyboardInterrupt: 

# Generate data for sample pairs

In [ ]:
features_tb, labels_tb, pnl_metadata_tb = generate_training_data(
        data=data_sampled,
        training_len=500,
        test_len=120,
        calculate_label=True,
        verbose=False
    )

# Write data out

In [ ]:
combined = pd.merge(features_tb, labels_tb, how='left', on=['Date', 'Ticker_P1','Ticker_P2']).reset_index(drop=True)

In [ ]:
combined = pd.merge(combined, pnl_metadata_tb[['Date', 'Ticker_P1','Ticker_P2', 'trade_executions']], how='left', on=['Date', 'Ticker_P1','Ticker_P2'])

In [ ]:
combined.columns

In [ ]:
with open('Data/spotcheckout_output.pkl','wb') as file:
    pickle.dump(combined, file)